In [1]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install -q condacolab
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

^C


'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [8]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

### Create Folders

from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

TRAIN_START_DATE = '2010-01-04'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-05-05'

tickers = ['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY']

df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = tickers).fetch_data()

df.sort_values(['date','tic'],ignore_index=True).head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (30213, 8)


,date,open,high,low,close,volume,tic,day
0,2010-01-04,33.580002,34.020000,33.450001,25.488928,7567500,XLB,0
1,2010-01-04,57.919998,58.810001,57.790001,38.080162,16928400,XLE,0
2,2010-01-04,11.795288,11.965881,11.770918,9.385947,92942347,XLF,0
3,2010-01-04,28.090000,28.320000,27.959999,21.880188,7471500,XLI,0
4,2010-01-04,23.139999,23.290001,23.100000,19.150221,8449400,XLK,0


In [11]:
#split data ticker wise
df_XLB = df[df['tic']=='XLB'][['date','tic','close']]
df_XLE = df[df['tic']=='XLE'][['date','tic','close']]
df_XLF = df[df['tic']=='XLF'][['date','tic','close']]
df_XLI = df[df['tic']=='XLI'][['date','tic','close']]
df_XLK = df[df['tic']=='XLK'][['date','tic','close']]
df_XLP = df[df['tic']=='XLP'][['date','tic','close']]
df_XLU = df[df['tic']=='XLU'][['date','tic','close']]
df_XLV = df[df['tic']=='XLV'][['date','tic','close']]
df_XLY = df[df['tic']=='XLY'][['date','tic','close']]

In [14]:
# Dlinear
D = pd.DataFrame({'XLB':df_XLB['close'].values,
                    'XLE':df_XLE['close'].values,
                    'XLF':df_XLF['close'].values,
                    'XLI':df_XLI['close'].values,
                    'XLK':df_XLK['close'].values,
                    'XLP':df_XLP['close'].values,
                    'XLU':df_XLU['close'].values,
                    'XLV':df_XLV['close'].values,
                    'XLY':df_XLY['close'].values},index=df_XLB['date'].values)

D

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
2010-01-04,25.488928,38.080162,9.385947,21.880188,19.150221,18.718464,19.481915,25.213638,25.227169
2010-01-05,25.571354,38.390972,9.558456,21.957457,19.125528,18.725483,19.249992,24.966366,25.319664
2010-01-06,26.005892,38.850708,9.577625,22.003801,18.911566,18.711449,19.362823,25.221615,25.353292
2010-01-07,25.803612,38.792439,9.782079,22.243311,18.837500,18.711449,19.275064,25.309351,25.563526
2010-01-08,26.163239,39.044956,9.724579,22.598709,18.960951,18.648277,19.256264,25.349237,25.555117
...,...,...,...,...,...,...,...,...,...
2023-04-28,80.550003,85.129997,33.169998,100.000000,150.830002,77.440002,68.980003,133.529999,147.860001
2023-05-01,80.510002,84.169998,33.080002,100.540001,150.919998,77.500000,69.099998,134.220001,146.399994
2023-05-02,79.730003,80.510002,32.330002,99.489998,149.509995,77.290001,68.260002,133.639999,146.550003
2023-05-03,78.849998,78.989998,31.959999,99.110001,148.250000,76.660004,68.070000,133.460007,145.389999


In [15]:
D.to_csv('D.csv')